Proof that the Python function matches the C implementation

In [ ]:
from hashlib import shake_256
from ctypes import c_int8

# combine the message hex stirng and salt, and return it as bytes
# example: message: "CAFE1337" salt:10 -> return value: b"CAFE13370A" 
def message_convert(m:str, salt:int) -> bytes:
    m = m+"%.2x"%(salt)
    b = bytes.fromhex(m) # convert from hex string to bytes
    return b

def print_gest(gest_hex:str):
    for r in range(len(gest_hex)):
        if(r%16 == 0):
            print("%3d " % (r>>4), end='')

        print("%s" % (gest_hex[r]), end='')
        if(r & 0x7 == 7):
            print(" ", end='')
        if(r & 0xf == 15):
            print()

def little_endian32(m:str):
    t = m[6:8] + m[4:6] + m[2:4] + m[0:2] 
    return t

def uint4_to_int4(a):
    a = a - ((a >> 3) & 1)*16
    return a

def cbd2(gest:str) -> list:
    d = []
    for k in range(int(len(gest)/8)):
        t = int(little_endian32(gest[k*8:k*8+8]), 16) & 0xFFFFFFFF
        # print(hex(t))
        t = (t & 0x55555555) + ((t >> 1) & 0x55555555)
        # this is the weird little_endian thing I did for the cbd module that works
        # shift = [24,28,16,20,8,12,0,4] 
        for i in range(8):
            a = (t >> (4*i  )) & 0x3
            b = (t >> (4*i+2)) & 0x3
            e = uint4_to_int4((a - b) & 0xf)
            d.append(e)
            # print((a - b) & 0xf,end=' ')
    return d

m = '02030405060708090a0b0c0d0e0f101112131415161718191a1b1c1d1e1f2021'
mm = message_convert(m,0)
h = shake_256(mm)
gest = h.digest(64*2)
gest_hex = gest.hex()
print(gest_hex[:15])
# print_gest(gest_hex)

gest_int = int(gest_hex,16)
c = cbd2(gest_hex)
print(c)
# for i in range(len(c)):
#     print("%x" %(c[i]),end=' ')
#     if(i%8==7):
#         print()

output = [0x0, 0x0, 0x1, 0x1, 0x0, 0x2, 0xf, 0x1,
0x0, 0x1, 0xf, 0x0, 0x0, 0x1, 0x1, 0x0,
0x0, 0x0, 0xf, 0x1, 0x0, 0x0, 0x0, 0x0,
0x1, 0x0, 0xf, 0xf, 0x1, 0x0, 0x0, 0xf,
0x0, 0xf, 0x1, 0x0, 0x2, 0xe, 0xe, 0x0, 
0x0, 0x0, 0xf, 0x1, 0x0, 0xf, 0x2, 0x0,
0x0, 0x1, 0x0, 0x1, 0x0, 0x0, 0x0, 0x0,
0xf, 0x1, 0x0, 0x1, 0xf, 0xf, 0x1, 0xf,
0x0, 0xf, 0xf, 0x1, 0xf, 0x0, 0x0, 0x1, 
0x1, 0x0, 0x1, 0x0, 0x2, 0x0, 0xe, 0xe,
0x2, 0xf, 0xf, 0x1, 0xf, 0x1, 0x1, 0x1,
0xf, 0xf, 0x1, 0xe, 0x0, 0x0, 0xf, 0xf,
0xf, 0x0, 0x1, 0x0, 0x0, 0xf, 0x0, 0xf, 
0xf, 0x0, 0x0, 0x0, 0xf, 0x0, 0x2, 0x1,
0x1, 0xf, 0x1, 0x1, 0xe, 0x1, 0x0, 0xe,
0x0, 0xf, 0xf, 0x0, 0x1, 0x1, 0x1, 0x0,
0x0, 0x1, 0x0, 0x1, 0x1, 0x1, 0x1, 0xf,
0x2, 0xf, 0x0, 0x0, 0x1, 0x1, 0xf, 0x1, 
0x0, 0xf, 0x1, 0x2, 0x0, 0xf, 0xf, 0xf,
0x1, 0x1, 0x2, 0x0, 0x1, 0x1, 0x0, 0x0,
0x0, 0x1, 0x1, 0x0, 0xf, 0x0, 0xf, 0x1,
0x1, 0xe, 0x1, 0x0, 0xf, 0x1, 0x0, 0x1,
0xf, 0xf, 0x0, 0x0, 0x1, 0x0, 0xf, 0xf, 
0x1, 0xf, 0xf, 0xf, 0x0, 0x1, 0x1, 0xf,
0xf, 0xe, 0x0, 0xf, 0xf, 0x1, 0x0, 0xf,
0x1, 0xf, 0x0, 0x0, 0x1, 0x0, 0x1, 0x0,
0x0, 0xf, 0x0, 0x0, 0xf, 0xf, 0xf, 0x1, 
0x0, 0x0, 0xf, 0xe, 0xf, 0x1, 0x1, 0x0,
0x2, 0x2, 0x0, 0x1, 0xf, 0x0, 0x0, 0xf,
0x2, 0x0, 0x0, 0x0, 0x0, 0x1, 0xf, 0x0,
0x0, 0xf, 0xf, 0xf, 0x2, 0xf, 0x1, 0x0,
0x1, 0x1, 0xf, 0x0, 0x0, 0x0, 0x0, 0xf]
for i in range(len(output)):
    n = uint4_to_int4(output[i])
    if(n != c[i]):
        print("proc")

print("[+] script done")

0ab7301db60e265
[0, 0, 1, 1, 0, 2, -1, 1, 0, 1, -1, 0, 0, 1, 1, 0, 0, 0, -1, 1, 0, 0, 0, 0, 1, 0, -1, -1, 1, 0, 0, -1, 0, -1, 1, 0, 2, -2, -2, 0, 0, 0, -1, 1, 0, -1, 2, 0, 0, 1, 0, 1, 0, 0, 0, 0, -1, 1, 0, 1, -1, -1, 1, -1, 0, -1, -1, 1, -1, 0, 0, 1, 1, 0, 1, 0, 2, 0, -2, -2, 2, -1, -1, 1, -1, 1, 1, 1, -1, -1, 1, -2, 0, 0, -1, -1, -1, 0, 1, 0, 0, -1, 0, -1, -1, 0, 0, 0, -1, 0, 2, 1, 1, -1, 1, 1, -2, 1, 0, -2, 0, -1, -1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, -1, 2, -1, 0, 0, 1, 1, -1, 1, 0, -1, 1, 2, 0, -1, -1, -1, 1, 1, 2, 0, 1, 1, 0, 0, 0, 1, 1, 0, -1, 0, -1, 1, 1, -2, 1, 0, -1, 1, 0, 1, -1, -1, 0, 0, 1, 0, -1, -1, 1, -1, -1, -1, 0, 1, 1, -1, -1, -2, 0, -1, -1, 1, 0, -1, 1, -1, 0, 0, 1, 0, 1, 0, 0, -1, 0, 0, -1, -1, -1, 1, 0, 0, -1, -2, -1, 1, 1, 0, 2, 2, 0, 1, -1, 0, 0, -1, 2, 0, 0, 0, 0, 1, -1, 0, 0, -1, -1, -1, 2, -1, 1, 0, 1, 1, -1, 0, 0, 0, 0, -1]
[+] script done


In [ ]:
from hashlib import shake_256

def cbd2_hash(m:str, salt:int):
    m1 = message_convert(m,salt)
    h = shake_256(m1)
    gest = h.digest(64*2)
    gest_hex = gest.hex()
    c = cbd2(gest_hex)
    return c

m = '02030405060708090a0b0c0d0e0f101112131415161718191a1b1c1d1e1f2021'
c = cbd2_hash(m,0)
print(c)


[0, 0, 1, 1, 0, 2, -1, 1, 0, 1, -1, 0, 0, 1, 1, 0, 0, 0, -1, 1, 0, 0, 0, 0, 1, 0, -1, -1, 1, 0, 0, -1, 0, -1, 1, 0, 2, -2, -2, 0, 0, 0, -1, 1, 0, -1, 2, 0, 0, 1, 0, 1, 0, 0, 0, 0, -1, 1, 0, 1, -1, -1, 1, -1, 0, -1, -1, 1, -1, 0, 0, 1, 1, 0, 1, 0, 2, 0, -2, -2, 2, -1, -1, 1, -1, 1, 1, 1, -1, -1, 1, -2, 0, 0, -1, -1, -1, 0, 1, 0, 0, -1, 0, -1, -1, 0, 0, 0, -1, 0, 2, 1, 1, -1, 1, 1, -2, 1, 0, -2, 0, -1, -1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, -1, 2, -1, 0, 0, 1, 1, -1, 1, 0, -1, 1, 2, 0, -1, -1, -1, 1, 1, 2, 0, 1, 1, 0, 0, 0, 1, 1, 0, -1, 0, -1, 1, 1, -2, 1, 0, -1, 1, 0, 1, -1, -1, 0, 0, 1, 0, -1, -1, 1, -1, -1, -1, 0, 1, 1, -1, -1, -2, 0, -1, -1, 1, 0, -1, 1, -1, 0, 0, 1, 0, 1, 0, 0, -1, 0, 0, -1, -1, -1, 1, 0, 0, -1, -2, -1, 1, 1, 0, 2, 2, 0, 1, -1, 0, 0, -1, 2, 0, 0, 0, 0, 1, -1, 0, 0, -1, -1, -1, 2, -1, 1, 0, 1, 1, -1, 0, 0, 0, 0, -1]


// in the C implementation
```c
int main(){
    poly *r = malloc(sizeof(poly));
    const uint8_t buf[] = { 0x0a, 0xb7, 0x30, 0x1d, 0xb6, 0x0e, 0x26, 0x5b, 
                        0x95, 0x1e, 0xaf, 0x5f, 0x6b, 0x88, 0x67, 0xe0, 
                        0xd9, 0x9b, 0xc3, 0x9c, 0xa0, 0x2d, 0x4a, 0x03, 
                        0x29, 0x1a, 0x5f, 0x9a, 0x1e, 0x20, 0x88, 0xe2, 
                        0xef, 0x1e, 0xf4, 0x2f, 0xf1, 0x51, 0x63, 0xcc, 
                        0xe3, 0x24, 0x1d, 0x71, 0xed, 0xc1, 0xa9, 0x8d, 
                        0x5e, 0x5b, 0x86, 0xda, 0x0d, 0x9f, 0x98, 0xb3, 
                        0xeb, 0x1b, 0x7c, 0xcf, 0xdf, 0xf8, 0x77, 0x67, 
                        0x7f, 0x29, 0x71, 0x8b, 0x43, 0x5a, 0xb7, 0x24, 
                        0xd5, 0x37, 0xe6, 0x8e, 0x77, 0x93, 0x27, 0xaf, 
                        0x29, 0x61, 0x68, 0x2e, 0xc7, 0x5b, 0x1e, 0xba, 
                        0x8e, 0x99, 0x67, 0xe4, 0x81, 0xee, 0x70, 0x47, 
                        0xcd, 0xd5, 0xbe, 0xea, 0xd2, 0x56, 0x07, 0xf1, 
                        0x89, 0x06, 0xdd, 0x7d, 0x90, 0xcd, 0xbd, 0x57, 
                        0x33, 0x76, 0xa4, 0x8f, 0xa3, 0x9a, 0x79, 0x6d, 
                        0x49, 0x8e, 0x43, 0x97, 0x21, 0xae, 0x6f, 0x89 };
    cbd2(r, buf);
}
output = {0x0, 0x0, 0x1, 0x1, 0x0, 0x2, 0xf, 0x1,
0x0, 0x1, 0xf, 0x0, 0x0, 0x1, 0x1, 0x0,
0x0, 0x0, 0xf, 0x1, 0x0, 0x0, 0x0, 0x0,
0x1, 0x0, 0xf, 0xf, 0x1, 0x0, 0x0, 0xf,
0x0, 0xf, 0x1, 0x0, 0x2, 0xe, 0xe, 0x0, 
0x0, 0x0, 0xf, 0x1, 0x0, 0xf, 0x2, 0x0,
0x0, 0x1, 0x0, 0x1, 0x0, 0x0, 0x0, 0x0,
0xf, 0x1, 0x0, 0x1, 0xf, 0xf, 0x1, 0xf,
0x0, 0xf, 0xf, 0x1, 0xf, 0x0, 0x0, 0x1, 
0x1, 0x0, 0x1, 0x0, 0x2, 0x0, 0xe, 0xe,
0x2, 0xf, 0xf, 0x1, 0xf, 0x1, 0x1, 0x1,
0xf, 0xf, 0x1, 0xe, 0x0, 0x0, 0xf, 0xf,
0xf, 0x0, 0x1, 0x0, 0x0, 0xf, 0x0, 0xf, 
0xf, 0x0, 0x0, 0x0, 0xf, 0x0, 0x2, 0x1,
0x1, 0xf, 0x1, 0x1, 0xe, 0x1, 0x0, 0xe,
0x0, 0xf, 0xf, 0x0, 0x1, 0x1, 0x1, 0x0,
0x0, 0x1, 0x0, 0x1, 0x1, 0x1, 0x1, 0xf,
0x2, 0xf, 0x0, 0x0, 0x1, 0x1, 0xf, 0x1, 
0x0, 0xf, 0x1, 0x2, 0x0, 0xf, 0xf, 0xf,
0x1, 0x1, 0x2, 0x0, 0x1, 0x1, 0x0, 0x0,
0x0, 0x1, 0x1, 0x0, 0xf, 0x0, 0xf, 0x1,
0x1, 0xe, 0x1, 0x0, 0xf, 0x1, 0x0, 0x1,
0xf, 0xf, 0x0, 0x0, 0x1, 0x0, 0xf, 0xf, 
0x1, 0xf, 0xf, 0xf, 0x0, 0x1, 0x1, 0xf,
0xf, 0xe, 0x0, 0xf, 0xf, 0x1, 0x0, 0xf,
0x1, 0xf, 0x0, 0x0, 0x1, 0x0, 0x1, 0x0,
0x0, 0xf, 0x0, 0x0, 0xf, 0xf, 0xf, 0x1, 
0x0, 0x0, 0xf, 0xe, 0xf, 0x1, 0x1, 0x0,
0x2, 0x2, 0x0, 0x1, 0xf, 0x0, 0x0, 0xf,
0x2, 0x0, 0x0, 0x0, 0x0, 0x1, 0xf, 0x0,
0x0, 0xf, 0xf, 0xf, 0x2, 0xf, 0x1, 0x0,
0x1, 0x1, 0xf, 0x0, 0x0, 0x0, 0x0, 0xf}
```